In [1]:
#Setup the enviroment on colab
#Google drive should be mounted to access folder with deduplicated images ('dedup')
#Two folders model_checkpoints/ and network_info/ must be located on Google drive
!unzip /content/drive/MyDrive/FetReg.zip
!unzip /content/FetReg2021_Task1_Segmentation.zip

!mkdir label
!mkdir /content/tensor_board_logs/


sh = """
for filename in /content/Video*; do
  echo $filename
  cp $filename/labels/* /content/label
done
"""
with open('script.sh', 'w') as file:
  file.write(sh)

!bash script.sh

!pip install -q -U albumentations
!echo "$(pip freeze | grep albumentations) is successfully installed"

!pip install iterative-stratification
!pip install -U segmentation-models


!pip install xgboost

Archive:  /content/drive/MyDrive/FetReg.zip
  inflating: EndoVis_FetReg_Rules.pdf  
  inflating: FetReg2021_Task1_Segmentation.zip  
Archive:  /content/FetReg2021_Task1_Segmentation.zip
   creating: Video001/
   creating: Video001/images/
 extracting: Video001/images/Video001_frame00500.png  
 extracting: Video001/images/Video001_frame01250.png  
 extracting: Video001/images/Video001_frame02785.png  
 extracting: Video001/images/Video001_frame02807.png  
 extracting: Video001/images/Video001_frame02830.png  
 extracting: Video001/images/Video001_frame02852.png  
 extracting: Video001/images/Video001_frame03081.png  
 extracting: Video001/images/Video001_frame03701.png  
 extracting: Video001/images/Video001_frame03706.png  
 extracting: Video001/images/Video001_frame03840.png  
 extracting: Video001/images/Video001_frame03863.png  
 extracting: Video001/images/Video001_frame03888.png  
 extracting: Video001/images/Video001_frame03911.png  
 extracting: Video001/images/Video001_frame039

In [2]:
import sys
sys.path.append('/content/drive/MyDrive/repos/image_processing_pipeline')


import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard
from tensorflow.keras.optimizers import Adam
from dataclasses import dataclass
from runner import *
from trainer import *
from initial_parser import *
from splitter import *
from data_provider import *
from custom_models import *
from custom_metrics import *
from custom_losses import *
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
import os
import random
from tensorflow.keras.layers.experimental import preprocessing 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import albumentations as A
import numpy as np
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from keras.layers import BatchNormalization
from keras.layers.core import SpatialDropout2D, Activation
from tensorflow.keras.applications import EfficientNetB0
from keras import backend as K
from keras.layers.merge import concatenate
from keras.utils.data_utils import get_file
from tensorflow import keras
import segmentation_models as sm
import cv2
import datetime
from segmentation_models.losses import bce_jaccard_loss
import pickle
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from ensembling import *
from settings import *


Segmentation Models: using `keras` framework.


In [3]:
def main(model_id):
  print(model_id)
  cur_params = params_dict[str(model_id)]
  input_test_file = cur_params['test_file']
  input_path_image = cur_params['data_path']
  input_path_mask = cur_params['label_path']
  input_img_size = cur_params['img_size']
  input_classes_cnt = cur_params['class_count']
  input_class_num = cur_params['class_num']
  input_test_size = cur_params['second_layer_test_size']
  input_final_test_path = cur_params['final_test_path']
  input_second_layer_model_path = cur_params['second_layer_model_path']
  input_second_layer_model_obj = cur_params['second_layer_model_obj']
  input_nn_zoo = cur_params['nn_zoo']
    
  with open(input_test_file, 'rb') as f:
    data = pickle.load(f)
  train_ensemble_ds, test =  train_test_split(data, test_size=input_test_size, random_state=42)
  with open(input_final_test_path, 'wb') as f:
      pickle.dump(test, f)
  parser_obj = parser_imseg_one_class_noclahe(input_path_image, input_path_mask, input_img_size, input_classes_cnt, input_class_num)
  ensemble_trainer_obj = ensemble_trainer(parser = parser_obj, img_size = input_img_size, nn_zoo = input_nn_zoo)
  ensemble_trainer_obj.train(second_layer_model_obj = input_second_layer_model_obj, second_layer_model_path = input_second_layer_model_path, second_layer_train_ds = train_ensemble_ds)
  

In [4]:
main(model_id = 'ensemble')

ensemble


/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
